#Entrada

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
# To add wait time between requests
import time

In [ ]:
os.environ['TOKEN'] = '...token...'

In [ ]:
def auth():
    return os.getenv('TOKEN')

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

#Tweets

In [ ]:
days = ["%.2d" % i for i in range(1,28)]
tomorrow = ["%.2d" % i for i in range(2,29)]

fevereiro = {}
keyword = "(Snow OR Wind OR Rain OR Storm OR Sunny OR Rainbow OR Cloudy OR Blizzard OR Tornado OR Thunderstorm) place_country:US lang:en -is:retweet -is:reply -is:quote"

for day in range(0,27):
  #Inputs for tweets
  bearer_token = auth()
  headers = create_headers(bearer_token)

  start_list =    ['2022-02-'+days[day]+'T00:00:00.000Z',
                  '2022-02-'+days[day]+'T01:00:00.000Z',
                  '2022-02-'+days[day]+'T02:00:00.000Z',
                  '2022-02-'+days[day]+'T03:00:00.000Z',
                  '2022-02-'+days[day]+'T04:00:00.000Z',
                  '2022-02-'+days[day]+'T05:00:00.000Z',
                  '2022-02-'+days[day]+'T06:00:00.000Z',
                  '2022-02-'+days[day]+'T07:00:00.000Z',
                  '2022-02-'+days[day]+'T08:00:00.000Z',
                  '2022-02-'+days[day]+'T09:00:00.000Z',
                  '2022-02-'+days[day]+'T10:00:00.000Z',
                  '2022-02-'+days[day]+'T11:00:00.000Z',
                  '2022-02-'+days[day]+'T12:00:00.000Z',
                  '2022-02-'+days[day]+'T13:00:00.000Z',
                  '2022-02-'+days[day]+'T14:00:00.000Z',
                  '2022-02-'+days[day]+'T15:00:00.000Z',
                  '2022-02-'+days[day]+'T16:00:00.000Z',
                  '2022-02-'+days[day]+'T17:00:00.000Z',
                  '2022-02-'+days[day]+'T18:00:00.000Z',
                  '2022-02-'+days[day]+'T19:00:00.000Z',
                  '2022-02-'+days[day]+'T20:00:00.000Z',
                  '2022-02-'+days[day]+'T21:00:00.000Z',
                  '2022-02-'+days[day]+'T22:00:00.000Z',
                  '2022-02-'+days[day]+'T23:00:00.000Z',
                  ]

  end_list =      ['2022-02-'+days[day]+'T01:00:00.000Z',
                  '2022-02-'+days[day]+'T02:00:00.000Z',
                  '2022-02-'+days[day]+'T03:00:00.000Z',
                  '2022-02-'+days[day]+'T04:00:00.000Z',
                  '2022-02-'+days[day]+'T05:00:00.000Z',
                  '2022-02-'+days[day]+'T06:00:00.000Z',
                  '2022-02-'+days[day]+'T07:00:00.000Z',
                  '2022-02-'+days[day]+'T08:00:00.000Z',
                  '2022-02-'+days[day]+'T09:00:00.000Z',
                  '2022-02-'+days[day]+'T10:00:00.000Z',
                  '2022-02-'+days[day]+'T11:00:00.000Z',
                  '2022-02-'+days[day]+'T12:00:00.000Z',
                  '2022-02-'+days[day]+'T13:00:00.000Z',
                  '2022-02-'+days[day]+'T14:00:00.000Z',
                  '2022-02-'+days[day]+'T15:00:00.000Z',
                  '2022-02-'+days[day]+'T16:00:00.000Z',
                  '2022-02-'+days[day]+'T17:00:00.000Z',
                  '2022-02-'+days[day]+'T18:00:00.000Z',
                  '2022-02-'+days[day]+'T19:00:00.000Z',
                  '2022-02-'+days[day]+'T20:00:00.000Z',
                  '2022-02-'+days[day]+'T21:00:00.000Z',
                  '2022-02-'+days[day]+'T22:00:00.000Z',
                  '2022-02-'+days[day]+'T23:00:00.000Z',
                  '2022-02-'+tomorrow[day]+'T00:00:00.000Z']
   

  max_results = 500

  #Total number of tweets we collected from the loop
  total_tweets = 0

  # Create file
  csvFile = open("data_22_02_"+days[day]+".csv", "a", newline="", encoding='utf-8')
  csvWriter = csv.writer(csvFile)

  #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
  csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
  csvFile.close()

  for i in range(0,len(start_list)):

      # Inputs
      count = 0 # Counting tweets per time period
      max_count = 100 # Max tweets per time period
      flag = True
      next_token = None
      
      # Check if flag is true
      while flag:
          # Check if max_count reached
          if count >= max_count:
              break
          print("-------------------")
          print("Token: ", next_token)
          url = create_url(keyword, start_list[i],end_list[i], max_results)
          json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
          result_count = json_response['meta']['result_count']

          if 'next_token' in json_response['meta']:
              # Save the token to use for next call
              next_token = json_response['meta']['next_token']
              print("Next Token: ", next_token)
              if result_count is not None and result_count > 0 and next_token is not None:
                  print("Start Date: ", start_list[i])
                  append_to_csv(json_response, "data_22_02_"+days[day]+".csv")
                  count += result_count
                  total_tweets += result_count
                  print("Total # of Tweets added: ", total_tweets)
                  print("-------------------")
                  time.sleep(5)                
          # If no next token exists
          else:
              if result_count is not None and result_count > 0:
                  print("-------------------")
                  print("Start Date: ", start_list[i])
                  append_to_csv(json_response, "data_22_02_"+days[day]+".csv")
                  count += result_count
                  total_tweets += result_count
                  print("Total # of Tweets added: ", total_tweets)
                  print("-------------------")
                  time.sleep(5)
              
              #Since this is the final request, turn flag to false to move to the next time period.
              flag = False
              next_token = None
          time.sleep(5)

  print('\n')
  print("******************")
  print('2022-02-'+ days[day])
  fevereiro_data = '2022-02-'+ days[day]
  fevereiro[fevereiro_data] = total_tweets
  print("Total number of results: ", total_tweets)
  print("******************")
  print('\n')

In [ ]:
days = ["%.2d" % i for i in range(28,29)]
tomorrow = ["%.2d" % i for i in range(1,2)]


for day in range(0,1):
  #Inputs for tweets
  bearer_token = auth()
  headers = create_headers(bearer_token)

  start_list =    ['2022-02-'+days[day]+'T00:00:00.000Z',
                  '2022-02-'+days[day]+'T01:00:00.000Z',
                  '2022-02-'+days[day]+'T02:00:00.000Z',
                  '2022-02-'+days[day]+'T03:00:00.000Z',
                  '2022-02-'+days[day]+'T04:00:00.000Z',
                  '2022-02-'+days[day]+'T05:00:00.000Z',
                  '2022-02-'+days[day]+'T06:00:00.000Z',
                  '2022-02-'+days[day]+'T07:00:00.000Z',
                  '2022-02-'+days[day]+'T08:00:00.000Z',
                  '2022-02-'+days[day]+'T09:00:00.000Z',
                  '2022-02-'+days[day]+'T10:00:00.000Z',
                  '2022-02-'+days[day]+'T11:00:00.000Z',
                  '2022-02-'+days[day]+'T12:00:00.000Z',
                  '2022-02-'+days[day]+'T13:00:00.000Z',
                  '2022-02-'+days[day]+'T14:00:00.000Z',
                  '2022-02-'+days[day]+'T15:00:00.000Z',
                  '2022-02-'+days[day]+'T16:00:00.000Z',
                  '2022-02-'+days[day]+'T17:00:00.000Z',
                  '2022-02-'+days[day]+'T18:00:00.000Z',
                  '2022-02-'+days[day]+'T19:00:00.000Z',
                  '2022-02-'+days[day]+'T20:00:00.000Z',
                  '2022-02-'+days[day]+'T21:00:00.000Z',
                  '2022-02-'+days[day]+'T22:00:00.000Z',
                  '2022-02-'+days[day]+'T23:00:00.000Z',
                  ]

  end_list =      ['2022-02-'+days[day]+'T01:00:00.000Z',
                  '2022-02-'+days[day]+'T02:00:00.000Z',
                  '2022-02-'+days[day]+'T03:00:00.000Z',
                  '2022-02-'+days[day]+'T04:00:00.000Z',
                  '2022-02-'+days[day]+'T05:00:00.000Z',
                  '2022-02-'+days[day]+'T06:00:00.000Z',
                  '2022-02-'+days[day]+'T07:00:00.000Z',
                  '2022-02-'+days[day]+'T08:00:00.000Z',
                  '2022-02-'+days[day]+'T09:00:00.000Z',
                  '2022-02-'+days[day]+'T10:00:00.000Z',
                  '2022-02-'+days[day]+'T11:00:00.000Z',
                  '2022-02-'+days[day]+'T12:00:00.000Z',
                  '2022-02-'+days[day]+'T13:00:00.000Z',
                  '2022-02-'+days[day]+'T14:00:00.000Z',
                  '2022-02-'+days[day]+'T15:00:00.000Z',
                  '2022-02-'+days[day]+'T16:00:00.000Z',
                  '2022-02-'+days[day]+'T17:00:00.000Z',
                  '2022-02-'+days[day]+'T18:00:00.000Z',
                  '2022-02-'+days[day]+'T19:00:00.000Z',
                  '2022-02-'+days[day]+'T20:00:00.000Z',
                  '2022-02-'+days[day]+'T21:00:00.000Z',
                  '2022-02-'+days[day]+'T22:00:00.000Z',
                  '2022-02-'+days[day]+'T23:00:00.000Z',
                  '2022-03-'+tomorrow[day]+'T00:00:00.000Z']

  max_results = 500

  #Total number of tweets we collected from the loop
  total_tweets = 0

  # Create file
  csvFile = open("data_22_02_"+days[day]+".csv", "a", newline="", encoding='utf-8')
  csvWriter = csv.writer(csvFile)

  #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
  csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
  csvFile.close()

  for i in range(0,len(start_list)):

      # Inputs
      count = 0 # Counting tweets per time period
      max_count = 100 # Max tweets per time period
      flag = True
      next_token = None
      
      # Check if flag is true
      while flag:
          # Check if max_count reached
          if count >= max_count:
              break
          print("-------------------")
          print("Token: ", next_token)
          url = create_url(keyword, start_list[i],end_list[i], max_results)
          json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
          result_count = json_response['meta']['result_count']

          if 'next_token' in json_response['meta']:
              # Save the token to use for next call
              next_token = json_response['meta']['next_token']
              print("Next Token: ", next_token)
              if result_count is not None and result_count > 0 and next_token is not None:
                  print("Start Date: ", start_list[i])
                  append_to_csv(json_response, "data_22_02_"+days[day]+".csv")
                  count += result_count
                  total_tweets += result_count
                  print("Total # of Tweets added: ", total_tweets)
                  print("-------------------")
                  time.sleep(5)                
          # If no next token exists
          else:
              if result_count is not None and result_count > 0:
                  print("-------------------")
                  print("Start Date: ", start_list[i])
                  append_to_csv(json_response, "data_22_02_"+days[day]+".csv")
                  count += result_count
                  total_tweets += result_count
                  print("Total # of Tweets added: ", total_tweets)
                  print("-------------------")
                  time.sleep(5)
              
              #Since this is the final request, turn flag to false to move to the next time period.
              flag = False
              next_token = None
          time.sleep(5)

  print('\n')
  print("******************")
  print('2022-02-'+days[day])
  fevereiro_data = '2022-02-'+days[day]
  fevereiro[fevereiro_data] = total_tweets
  print("Total number of results: ", total_tweets)
  print("******************")
  print('\n')

In [ ]:
fevereiro_df = pd.DataFrame(list(fevereiro.items()), columns=['data', 'tweets'])
fevereiro_df